This notebook demonstrates some basic techniques for working with LIDAR data in Jupyter.

Resources:
- Breddels, Maarten. “Interactive 3D Visualization in Jupyter.” Presented at the SciPy 2018, February 4, 2018. [https://www.youtube.com/watch?v=hOKa8klJPyo](https://www.youtube.com/watch?v=hOKa8klJPyo).
- Carette, Mathieu. “Efficient and Interactive 3D Point Cloud Processing.” Presented at the FOSDEM 2018, February 4, 2018. [https://www.youtube.com/watch?v=1oXVDG6Iop0](https://www.youtube.com/watch?v=1oXVDG6Iop0).




In [87]:
from dotenv import load_dotenv, find_dotenv
from geoalchemy2 import Geometry, WKTElement
import geopandas as gpd
import ipyleaflet
import ipyvolume as ipv
import numpy as np
import pandas as pd
import pyproj
import pythreejs
from shapely.geometry import shape
from shapely.ops import transform
from sqlalchemy import *


In [2]:
# query postgis to retrieve points
load_dotenv(find_dotenv())
conn_vars = ['DEV_USER', 'DEV_PASSWORD', 'DEV_HOST', 'DEV_PORT', 'DEV_DB']
user, password, host, port, dbname = [os.getenv(var) for var in conn_vars]
conn_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'

In [3]:
engine = create_engine(conn_string)

In [60]:
m = ipyleaflet.Map(center=(40.712160, -74.007153), zoom=16)
dc = ipyleaflet.DrawControl()
m.add_control(dc)
m

Map(center=[40.71216, -74.007153], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [73]:
# get site bounds reprojected as epsg 2263
project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:2263')
)

map_bounds = dc.last_draw['geometry']
site_bounds = transform(project.transform, shape(map_bounds)).wkt
site_bounds


'POLYGON ((985871.2810432309 198158.5470172858, 986638.6577798544 198309.4430955068, 986600.0928350925 198611.8334662987, 985833.55255324 198530.890058656, 985871.2810432309 198158.5470172858))'

In [83]:
query = f'''
WITH patches AS (
    SELECT pa
    FROM
        doitt_pointcloud_v2017 pts
    WHERE
        PC_Intersects(
            pts.pa,
            ST_Buffer(
                ST_GeomFromText(
                    '{site_bounds}', 
                    2263
                ),
                15
            )
        )        
),
points AS (
    SELECT
        PC_Explode(pa)
        AS pt
    FROM
        patches
),
filtered_points AS ( 
    SELECT 
        PC_Get(pt, 'x') AS x,
        PC_Get(pt, 'y') AS y,
        PC_Get(pt, 'z') AS z,
        pt
    FROM
        points
        -- mtsthelens_rim r
    -- WHERE
    -- ST_Intersects (pt::geometry,
    -- ST_Buffer(r.geom, 15)) 
)
SELECT
    Geometry(pt) as geometry
FROM 
    filtered_points;
'''

In [86]:
# get geodataframe from postgis
gdf = gpd.read_postgis(query, engine, geom_col='geometry')
gdf['geometry']

0          POINT Z (985837.720 198099.580 8.270)
1          POINT Z (985837.830 198109.030 8.270)
2          POINT Z (985837.840 198135.410 9.550)
3          POINT Z (985837.960 198091.240 8.010)
4          POINT Z (985837.970 198115.620 8.430)
                           ...                  
188960    POINT Z (986632.960 198629.580 84.740)
188961    POINT Z (986633.020 198646.410 76.570)
188962    POINT Z (986633.030 198612.440 24.840)
188963    POINT Z (986633.040 198642.010 86.320)
188964    POINT Z (986633.040 198613.100 75.790)
Name: geometry, Length: 188965, dtype: geometry

In [ ]:
# visualize the results in ipyvolume


In [88]:
# simple example from Breddels 2018
N = 1000
x, y, z = np.random.normal(0, 1, (3, N))
fig = ipv.figure()
# fig.camera_control = 'pan'

control = pythreejs.OrbitControls(controlling=fig.camera)
fig.controls = control
# control.autoRotate = True
fig.render_continuous = True

scatter = ipv.scatter(x, y, z, marker='sphere')
ipv.show()

In [ ]:
scatter.geo = 'box'
scatter.x = x + 2
# ipv.save

In [ ]:
# process the results to get a "section"